In [1]:
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.multiprocessing as mp
from torchinfo import summary

rng = np.random.default_rng()

In [2]:
input_transform = transforms.Compose([
    transforms.Resize(64, interpolation=transforms.InterpolationMode.BICUBIC, antialias=True),
    transforms.ToTensor(),
])

In [3]:
training_data = datasets.EMNIST(root="Torch MNIST", split="digits", train=True,download=True, transform=input_transform)
test_data = datasets.EMNIST(root="Torch MNIST", split="digits", train=False,download=True, transform=input_transform)

In [4]:
batch_size = 2**12
train_dataloader = DataLoader(training_data, batch_size = batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size = batch_size, shuffle=True)

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [6]:
class AddGaussianNoise(nn.Module):
    def __init__(self, mean=0., std=1., *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.std = std
        self.mean = mean
        
    def forward(self, tensor):
        return tensor + torch.randn(tensor.size(), device=device) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [7]:
transform_sequence = torch.nn.Sequential(
    transforms.RandomAffine(15, translate=(0.1,0.1), scale=(0.9,1.1),interpolation=transforms.InterpolationMode.BILINEAR),
    AddGaussianNoise(mean=0.0,std=0.1)
)

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self,):
        super().__init__()
        self.conv_stack = nn.Sequential(OrderedDict([
            ("conv1", nn.Conv2d(1,8,5, padding=2)),
            ("act1", nn.LeakyReLU()),
            ("batch_norm1", nn.BatchNorm2d(8)),
            ("conv2", nn.Conv2d(8,8,5, padding=2)),
            ("act2", nn.LeakyReLU()),
            ("batch_norm2", nn.BatchNorm2d(8)),
            ("max_pool1", nn.MaxPool2d(4)),
            ("conv3", nn.Conv2d(8,16,3,padding=1)),
            ("act3", nn.LeakyReLU()),
            ("batch_norm3", nn.BatchNorm2d(16)),
            ("conv4", nn.Conv2d(16,16,3,padding=1)),
            ("act4", nn.LeakyReLU()),
            ("batch_norm4", nn.BatchNorm2d(16)),
            ("max_pool2", nn.MaxPool2d(2)),
        ]))
        self.linear_stack = nn.Sequential(OrderedDict([
            ("flatten", nn.Flatten()),
            ("dropout1", nn.Dropout(0.1)),
            ("linear1", nn.Linear(8*8*16,512)),
            ("act1", nn.LeakyReLU()),
            ("batch_norm1", nn.BatchNorm1d(512)),
            ("dropout2", nn.Dropout(0.1)),
            ("linear2", nn.Linear(512,64)),
            ("act2", nn.LeakyReLU()),
            ("batch_norm2", nn.BatchNorm1d(64)),
            ("linear3", nn.Linear(64,10)),
        ]))

    def forward(self, x):
        if self.training:
            x = transform_sequence(x)
        x = self.conv_stack(x)
        x = self.linear_stack(x)
        return x

In [9]:
model = NeuralNetwork().to(device)
summary(model, input_size=(batch_size, 1, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [4096, 10]                --
├─Sequential: 1-1                        [4096, 16, 8, 8]          --
│    └─Conv2d: 2-1                       [4096, 8, 64, 64]         208
│    └─LeakyReLU: 2-2                    [4096, 8, 64, 64]         --
│    └─BatchNorm2d: 2-3                  [4096, 8, 64, 64]         16
│    └─Conv2d: 2-4                       [4096, 8, 64, 64]         1,608
│    └─LeakyReLU: 2-5                    [4096, 8, 64, 64]         --
│    └─BatchNorm2d: 2-6                  [4096, 8, 64, 64]         16
│    └─MaxPool2d: 2-7                    [4096, 8, 16, 16]         --
│    └─Conv2d: 2-8                       [4096, 16, 16, 16]        1,168
│    └─LeakyReLU: 2-9                    [4096, 16, 16, 16]        --
│    └─BatchNorm2d: 2-10                 [4096, 16, 16, 16]        32
│    └─Conv2d: 2-11                      [4096, 16, 16, 16]        2,320
│    

In [10]:
loss_fn = nn.CrossEntropyLoss()
Optimiser = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)
model.train(True)

for epoch in range(0,1000):
    running_loss = 0.0
    for batch_num, batch_data in enumerate(train_dataloader):
        #print("Epoch {0} Batch {1}".format(epoch+1, batch_num+1))
        Optimiser.zero_grad()
        
        data, labels = batch_data
        labels = nn.functional.one_hot(labels, num_classes = 10).float()
        data = data.to(device)
        labels = labels.to(device)
        
        prediction = model(data)
        loss = loss_fn(prediction,labels)
        
        loss.backward()
        
        running_loss += loss.item()
        log_num = 1
        if log_num == 1:
            print(f'[{epoch + 1}, {batch_num + 1:5d}] loss: {running_loss:.3f}')
            running_loss = 0.0
        else:
            if batch_num % log_num == (log_num-1):
                print(f'[{epoch + 1}, {batch_num + 1:5d}] loss: {running_loss / log_num:.3f}')
                running_loss = 0.0
        print()
        Optimiser.step()

[1,     1] loss: 2.451
[1,     2] loss: 2.410
[1,     3] loss: 2.013
[1,     4] loss: 1.902
[1,     5] loss: 1.423
[1,     6] loss: 1.419
[1,     7] loss: 1.339
[1,     8] loss: 1.107
[1,     9] loss: 0.967
[1,    10] loss: 1.228
[1,    11] loss: 0.776
[1,    12] loss: 0.880
[1,    13] loss: 0.698
[1,    14] loss: 0.561
[1,    15] loss: 1.355
[1,    16] loss: 0.766
[1,    17] loss: 0.710
[1,    18] loss: 0.640
[1,    19] loss: 0.744
[1,    20] loss: 1.139
[1,    21] loss: 0.464
[1,    22] loss: 0.616
[1,    23] loss: 0.402
[1,    24] loss: 0.406
[1,    25] loss: 0.412
[1,    26] loss: 0.426
[1,    27] loss: 0.375
[1,    28] loss: 0.453
[1,    29] loss: 0.320
[1,    30] loss: 0.303
[1,    31] loss: 0.376
[1,    32] loss: 0.226
[1,    33] loss: 0.220
[1,    34] loss: 0.214
[1,    35] loss: 0.386
[1,    36] loss: 1.171
[1,    37] loss: 0.246
[1,    38] loss: 0.584
[1,    39] loss: 0.253
[1,    40] loss: 0.297
[1,    41] loss: 0.299
[1,    42] loss: 0.415
[1,    43] loss: 0.455
[1,    44] 

In [12]:
total = 0
correct = 0
model.eval()

with torch.no_grad():
    for loader_data in test_dataloader:
        data, labels = loader_data
        data = data.to(device)
        labels = labels.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print(1 - (correct/total))

0.002375000000000016
